In [136]:
import numpy as np
import pandas as pd

import lightgbm
import matplotlib.pylab as plt
import os

from sklearn.model_selection import train_test_split
import sklearn

Inserting all features from test into one dataframe

In [137]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)
        
        

       average_income_per_grunnkrets  distance_store_busstop  \
0                           561700.0              585.354489   
1                           555720.0              138.798366   
2                           495900.0               41.099519   
3                           521240.0               56.693017   
4                           585360.0              229.077959   
...                              ...                     ...   
12854                       505640.0              150.004125   
12855                       396640.0              322.573926   
12856                       549120.0              174.851316   
12857                       498660.0              121.372663   
12858                       579780.0              189.983143   

       distance_to_competitor  has_mall_or_not  plaace_h_1  plaace_h_2  \
0                    0.000000                1           1           1   
1                   34.547236                0           1           1   
2        

Repeat for test features

In [138]:
folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)
    

      average_income_per_grunnkrets  distance_store_busstop  \
0                          551600.0               25.630061   
1                          531240.0              111.736189   
2                          523300.0              404.825119   
3                          591380.0              317.141300   
4                          594020.0              101.273354   
...                             ...                     ...   
8572                       560700.0              364.034347   
8573                       593340.0              228.923169   
8574                       629100.0              136.232163   
8575                       590100.0              129.203732   
8576                       529980.0              212.494198   

      distance_to_competitor  has_mall_or_not  plaace_h_1  plaace_h_2  \
0                  70.085760                0           1           1   
1                 177.710439                0           1           1   
2                   0.00

Target train data:

In [139]:
target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

#log transform
target_train = np.log1p(target_train)

Light GBM model:

In [140]:
lgbm_model = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

In [141]:
#print(features_train)
#print(type(features_train))
print(len(features_train))
print(target_train)
print(type(target_train))
target_train = target_train
print(len(target_train))

12859
[2.94433371 3.21197205 2.83901998 ... 3.6693143  1.53514531 1.20237152]
<class 'numpy.ndarray'>
12859


In [142]:
lgbm_model.fit(features_train, target_train)
lgbm_predictions = lgbm_model.predict(features_test)

#Undo log transform
lgbm_predictions = np.expm1(lgbm_predictions)

c:\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


In [143]:
print(lgbm_predictions)

[6.50352858 5.53072994 6.57307854 ... 3.43605294 3.20013402 1.82200499]


In [144]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [145]:
predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = lgbm_predictions
predictions.to_csv("../predictions/lightGBM14.csv", index=False)
print(predictions)


                              id  predicted
0     914206820-914239427-717245   6.503529
1     916789157-916823770-824309   5.530730
2       913341082-977479363-2948   6.573079
3      889682582-889697172-28720   4.096673
4     997991699-998006945-417222   4.940493
...                          ...        ...
8572  917323003-917383529-844309   1.360800
8573  917353379-917411824-845904   3.089863
8574  917072302-917089248-833647   3.436053
8575  916960557-916993161-829908   3.200134
8576   987280891-972040746-45320   1.822005

[8577 rows x 2 columns]


In [146]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))


X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

lgbm_model2 = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

lgbm_model2.fit(X_train, y_train)
y_pred = lgbm_model2.predict(X_test)
# Calculate rmsle for a few example predictions 
y_true = y_test
n = len(features_train)
print('A couple of RMSLE scores computed over the train set')
print(f'Perfect prediction: {rmsle(y_true, y_true):.4f}')
print(f'Score: {rmsle(y_true, y_pred):.4f}')

print("Score: ",sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5)

c:\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


A couple of RMSLE scores computed over the train set
Perfect prediction: 0.0000
Score: 0.3412
Score:  0.3412001726773088
